In [1]:
#!pip install -q datasets transformers seqeval evaluate accelerate

In [2]:
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate

# %%
# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003")

C:\Users\binja\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:

# %%
# Define the label list
label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)
label_list

# %%
# Load the tokenizer and model
model_checkpoint = "prajjwal1/bert-mini"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

C:\Users\binja\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\binja\.cache\huggingface\hub\models--prajjwal1--bert-mini. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:

# %%
# Tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# %%
# Apply the tokenization and label alignment
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# %%
# Data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Map: 100%|██████████| 3453/3453 [00:00<00:00, 15251.56 examples/s]


In [12]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
)

# %%
# Load the evaluation metric
metric = evaluate.load("seqeval")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [15]:

training_args = TrainingArguments(
    output_dir="./ner_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [17]:

# %%
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.171000,0.169794,0.727634,0.770616,0.748508,0.955999
2,0.127700,0.130831,0.806171,0.826658,0.816286,0.966201
3,0.106700,0.121855,0.812919,0.836587,0.824583,0.967778


TrainOutput(global_step=2634, training_loss=0.20508806496835968, metrics={'train_runtime': 549.8154, 'train_samples_per_second': 76.613, 'train_steps_per_second': 4.791, 'total_flos': 37919633543190.0, 'train_loss': 0.20508806496835968, 'epoch': 3.0})

In [18]:

# %%
# Save the trained model and tokenizer
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")


('./ner_model\\tokenizer_config.json',
 './ner_model\\special_tokens_map.json',
 './ner_model\\vocab.txt',
 './ner_model\\added_tokens.json',
 './ner_model\\tokenizer.json')